In [ ]:
#Written using Python 3.9.7
#Author: icebreaker, May 2022

from time import sleep, time
from web3 import Web3
import numpy as np
import json
from scipy.stats import norm
from os.path import exists
from hexbytes import HexBytes
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
from textwrap import wrap
from collections import Counter


pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
INFURA_KEY = ps.getenv("INFURA_API_KEY")

ETHERSCAN_API_BASE_URL = "https://api.etherscan.io/api"

w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/{}".format(INFURA_KEY)))

In [ ]:
def findTargetTX(startBlock, blockStep, _targetEventTopicSignature, _fileNamePreFix):
    targetEventTopicSignature = _targetEventTopicSignature
    fileNamePreFix = _fileNamePreFix
    latestBlockNumber = startBlock
    currentBlockNumber = w3.eth.get_block_number()
    filteredTXs = []

    while latestBlockNumber < currentBlockNumber:
        eventFilter = w3.eth.filter(
            {
                "fromBlock": latestBlockNumber,
                "toBlock": latestBlockNumber + blockStep,
                #"address": targetContract,
                "topics": [targetEventTopicSignature],
            }
        )
        txLogs = w3.eth.get_filter_logs(eventFilter.filter_id)

        if len(txLogs):
            ts = pd.to_datetime((w3.eth.get_block(latestBlockNumber)["timestamp"]), unit='s').to_datetime64()
            print(
                datetime.now().strftime("%H:%M:%S"), 
                "Time: {} Covering Block {}-{}: Found {} {} Transfer events".format(
                    ts,
                    latestBlockNumber,
                    latestBlockNumber + blockStep,
                    len(txLogs),
                    fileNamePreFix
                )
            )
            idx = 0
            for log in txLogs:
                txnHash = log["transactionHash"]
                print("Getting TX: ",txnHash.hex(), " ", idx, " of", len(txLogs))

                txn = w3.eth.get_transaction(txnHash)
                gasLimit = txn["gas"]
                gasPrice = txn["gasPrice"]
                blockNumber = txn["blockNumber"]

                txnReceipt = w3.eth.get_transaction_receipt(txnHash)
                gasUsed = txnReceipt["gasUsed"]

                block = w3.eth.get_block(blockNumber)
                timestamp = block["timestamp"]


                try:
                    mainMethodID = txn.input[:10]
                    mainPayload = txn.input[10:]
                    mainPayloadTokenized = wrap(mainPayload,64)
                    mainOffset = int((int(mainPayloadTokenized[2],16) * 2) / 64)
                    mainLength = int(mainPayloadTokenized[mainOffset],16) * 2
                    gnosisDataInput = txn.input[((mainOffset + 1) * 64 ) + 10: ((mainOffset + 1) * 64) + mainLength + 10]
                    nestedMethodID = "0x" + gnosisDataInput[:8]
                    nestedPayload = gnosisDataInput[8:]

                    filteredTXs.append(
                    {
                            "txHash": txnHash.hex(),
                            "gasLimit": int(gasLimit),
                            "gasUsed": int(gasUsed),
                            "gasPrice": int(gasPrice),
                            "timestamp": timestamp,
                            "blockNumber": blockNumber,
                            "from": txn["from"],
                            "to": txn["to"],
                            'payloadTO' : '0x' + mainPayloadTokenized[0][24:],
                            'payloadValue' : '0x' + mainPayloadTokenized[1],
                            'payloadMethodID' : mainMethodID,
                            'payLoadOffset': mainOffset,
                            'payLoadLength': mainLength,
                            'nestedMethodID' : nestedMethodID,
                            'nestedPayload' : nestedPayload
                        }
                    )
                except:
                    print("error")
                idx += 1
            latestBlockNumber += blockStep

        # sleep to avoid getting rate limited
        sleep(0.001)
    # Save all the order fill data in a pickle file
    print("Found {} total tradExecs ", fileNamePreFix ," fills".format(len(filteredTXs)))
    return filteredTXs

In [ ]:
topicSig = '0x442e715f626346e8c54381002da614f62bee8d27386535b2521ec8540898556e' #transferExec

currentBlockNumber = w3.eth.get_block_number()
#Finding and scraping onchain tx's of interest takes approximately ~2 hours via infura
v2Events = findTargetTX(14710000, 10000, topicSig, 'tradeRebalExecs')

columnsFromScrape = ["txHash","gasLimit","gasUsed","gasPrice", "timestamp","blockNumber","from","to",'payloadTO','payloadValue','payloadMethodID', 'payLoadOffset', 'payLoadLength', 'nestedMethodID', 'nestedPayload']
finals = pd.DataFrame(v2Events, columns = columnsFromScrape)
finals['datetime'] = list(map(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3], finals["timestamp"]))

combinedFills = pd.concat([finals])
np.save("gnosisExecTXs", np.array(combinedFills), allow_pickle=True)
combinedFills.to_csv("combinedFills.to_csv")

In [ ]:
combinedFills = pd.read_csv("combinedFills.to_csv")
cols = combinedFills.columns

combinedFills = np.load("gnosisExecTXs.npy", allow_pickle=True)
combinedFills = pd.DataFrame(combinedFills)
combinedFills.columns = cols[1:]
combinedFills.reset_index()

from collections import Counter

#Counter(combinedFills['nestedMethodID'])
Counter(combinedFills['payloadTO'])

df1 = combinedFills.groupby('payloadTO').size().sort_values().reset_index(name='count')
df1.to_csv("topTo.csv")

#UNIWAPV2 Message Parser from Gnosis execTransfer Payload <- Used to debug, generalised transfer solution
def decodUniV2TXSentToken(pl):
    _pl =pl[32:]
    plw = wrap(_pl,64)
    sendToken = '0x' + plw[5][:40]
    return sendToken

def decodUniV2TXRecToken(pl):
    _pl =pl[32:]
    plw = wrap(_pl,64)
    receiveToken = '0x' + plw[6][:40]
    return [receiveToken]

def decodUniV2TXSentTokenAmount(pl):
    _pl =pl[32:]
    plw = wrap(_pl,64)
    amountSent = int((plw[9][:40]),16)  
    return [amountSent]

listOfProtocols = ['0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45' , '0x7d2768de32b0b80b7a3454c06bdac94a69ddc7a9', '0xc36442b4a4522e871399cd717abdd847ab11fe88', '0x1111111254fb6c44bac0bed2854e76f90643097d', '0x9008d19f58aabd9ed0d60971565aa8510560ab41']
uniswapTxDF = combinedFills[combinedFills['payloadTO'].isin(listOfProtocols)]
uniswapTxDF['sendToken'] = uniswapTxDF['nestedPayload'].apply(decodUniV2TXSentToken)
uniswapTxDF['receiveToken'] = uniswapTxDF['nestedPayload'].apply(decodUniV2TXRecToken)
uniswapTxDF['amountSent'] = uniswapTxDF['nestedPayload'].apply(decodUniV2TXSentTokenAmount)

uniswapTxDF[['amountSent'] ]

In [ ]:
#Download transfer data

transferTopicSignature = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
wethAdd = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

sellTransactions = []
transferEvents = []
idx = 0

for log in uniswapTxDF['txHash']:
    idx = idx + 1

    txnHash = log
    #txnHash = '0xb247c4d01e521fa4e5db280ef907b43aa5639e427d5183940a745adc168cd2e8'
    txDetails = w3.eth.get_transaction(txnHash)
    txReceipt = w3.eth.get_transaction_receipt(txnHash)
    txLogLen = len(txReceipt.logs)

    transferFound = False
    
    for logNested in range(0,txLogLen):
            #print("Working: " , log/txLogLen, "%")
        try:
            if txReceipt.logs[logNested].topics[0].hex() == transferTopicSignature:
                print("transferFound")
                tokenAddress = txReceipt.logs[logNested].address
                amount = int(txReceipt.logs[logNested].data,16)
                transferEvents.append([txnHash, txDetails.value, tokenAddress, amount])    
                print(idx, len(uniswapTxDF), txnHash, tokenAddress, amount)
                break 
        except:
            tokenAddress = txReceipt.logs[logNested].address
            amount = 0
            transferEvents.append([txnHash, txDetails.value, tokenAddress, amount])    
            print("ERROR", idx, len(uniswapTxDF), txnHash, tokenAddress, amount)

            break

    sleep(0.01)
    
transferUniV2Events = pd.DataFrame (transferEvents, columns = ['txnHash', 'value', 'tokenAddress', 'amount'])
transferUniV2Events.to_csv("transferEvents.csv")

In [ ]:
transferUniV2Events = pd.read_csv("transferEvents.csv")

#ADD IN Symbols and Decimals
uniqueTokens = transferUniV2Events['tokenAddress']
uniqueTokens = uniqueTokens.unique()

with open('./abi/erc20.abi') as json_file:
    erc20ABI = json.load(json_file)

metaData=[]
idx = 0 
lent = len(uniqueTokens)
for token in uniqueTokens:
    idx = 1 + idx
    print(idx, lent, token)
    try:

        contract = w3.eth.contract(w3.toChecksumAddress(token), abi=erc20ABI)
        symbol = contract.functions.symbol().call()

        metaData.append(
        {
            "tokenAddress": token,
            "decimals": contract.functions.decimals().call(),
            "symbol" : symbol
        })

    except:
        print("ERROR", idx, lent, token, )
        symbol = "MKR" #catch MKR's bytes32 symbol encoding

    sleep(0.1)


sellMetaDataDF = pd.DataFrame(data=metaData)
sellMetaDataDF.columns =['tokenAddress','sellDecimals','sellSymbol']

tradeExecs = pd.merge(transferUniV2Events, 
                     sellMetaDataDF, 
                     left_on ='tokenAddress',
                     right_on ='tokenAddress',
                     how ='left')

In [ ]:
uniswapTxDF = pd.merge(tradeExecs, 
                     uniswapTxDF, 
                     left_on ='txnHash',
                     right_on ='txHash',
                     how ='left')

tradeExecs = uniswapTxDF

#Get approx USD amount, do this by creating a primary key of sold token + date and then making it unique, 
# run the query (contract address + date) vs coingeckos API to get USD mark then join back to main DF
uniquePriceQueries = []
tradeExecs['datetime'] = list(map(lambda x: datetime.fromtimestamp(x).strftime('%d-%m-%Y'), tradeExecs["timestamp"]))
cols = ['tokenAddress', 'datetime']
tradeExecs['uniquePriceQueriesPK'] = tradeExecs[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [ ]:
def getPriceUSD(contractAdd, date):
    id = cg.get_coin_info_from_contract_address_by_id('ethereum',contractAdd).get('id')
    price = cg.get_coin_history_by_id(id, date).get('market_data').get('current_price').get('usd')
    return price
i = 0 

for priceQuery in (tradeExecs['uniquePriceQueriesPK'].unique()):
    contract = (priceQuery[:42])
    date = (priceQuery[-10:])

    try: 
        uniquePriceQueries.append({
        'uniquePriceQueriesPK' : priceQuery,
        'approxUSDMarkSell' : getPriceUSD(contract, date)
        })
    except:
        uniquePriceQueries.append({
        'uniquePriceQueriesPK' : priceQuery,
        'approxUSDMarkSell' : 0
        })
    print(i, "/", len(tradeExecs['uniquePriceQueriesPK'].unique()))
    i += 1
    sleep(.5) #run the query slow, because CG is not l33t

In [ ]:
#join prices to trades
tradeExecs['amount'] = tradeExecs['amount'].astype(float) / ( 10 ** tradeExecs['sellDecimals'])

priceQueryDF = pd.DataFrame(data=uniquePriceQueries)
priceQueryDF.columns =['uniquePriceQueriesPK', 'approxUSDMarkSell']
tradeExecs = pd.merge(tradeExecs, 
                     priceQueryDF, 
                     left_on ='uniquePriceQueriesPK',
                     right_on ='uniquePriceQueriesPK',
                     how ='left')
                
tradeExecs['totalTradeValue'] = tradeExecs['amount'] * tradeExecs['approxUSDMarkSell'] 

In [ ]:
tradeExecs['totalTradeValue'].sum()

In [ ]:
#Dump Numpy and CSV of tradeExec DF to disk
np.save("protocolActivityFinal" , np.array(tradeExecs), allow_pickle=True)
tradeExecs.to_csv("protocolActivityFinal.csv")